# Document scanning tool

### Imports

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import argparse
from elasticsearch import Elasticsearch
from dl_docs import *
from process_docs import *
from inverse_index import *

%load_ext autoreload
%autoreload 2

## Download docs from 3gpp

In [2]:
%%script false --no-raise-error
url = 'https://www.3gpp.org/ftp/Specs/2023-06'  # URL of the webpage to be parsed

# Download zip files
download_docs_from_3gpp(url, max_files=100)

  0%|          | 0/12 [00:02<?, ?it/s]


Max file limit reached. Returning 100 files.


100%|██████████| 100/100 [02:39<00:00,  1.60s/it]

Done!


## Create reverse index

### Extract text from docs

In [2]:
%%script false --no-raise-error
### Extract text from docs
# extract text
doc2txt()

sh: 1: Syntax error: "(" unexpected


### Create inverse index

In [14]:
# %%script false --no-raise-error
# create elasticsearch object
es = Elasticsearch("http://localhost:9200")

# Index the .txt files
index_files_to_es(es)

### Explore index

In [ ]:
%%script false --no-raise-error
# explore index
indices = es.indices.get_alias("*")
for idx in indices:
    print(idx)
    
index_name = "3gpp_docs"
# retrieve docs from index
# response = es.search(index=index_name, body={"query": {"match_all": {}}}, size=10)  # retrieve 10 documents

# for hit in response["hits"]["hits"]:
    # print(hit["_source"])  # print the source of the document

# count documents in index
doc_count = es.count(index=index_name)["count"]
print(f"Total documents in {index_name}: {doc_count}")

# get mapping
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

# search for a term
term = 'TS 22.101'
response = es.search(index=index_name, body={"query": {"match": {"content": term}}})

for hit in response["hits"]["hits"]:
    print(hit["_source"]['filename'])


### Search index

In [15]:
term = 'TS 22.101'
results = search_docs(es, term)
for result in results:
    print(f"Filename: {result['filename']}, URL: {result['url']}")

Filename: 21202-a31.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/21202-a31.txt
Filename: 22001-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22001-a00.txt
Filename: 22004-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22004-a00.txt
Filename: 22003-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22003-a00.txt
Filename: 22088-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22088-a00.txt
Filename: 22094-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22094-a00.txt
Filename: 22030-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22030-a00.txt
Filename: 22934-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22934-a00.txt
Filename: 22082-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22082-a00.txt
Filename: 22246-a00.txt, URL: file:///home/neil/docscan-philips/data/raw_docs/22246-a00.txt
